# Curso: Bioestatística — Experimento de Regressão Linear
## Autores: Sandro da Silva Camargo e Fernando Cardoso

**Problema**:Um experimento foi realizado para testar o efeito
da adubação nitrogenada (0, 100, 200 e 300 kg/ha de Adubo
Nitrogenado) na produção de milho (kg/parcela).


A base de dados está disponível [aqui](https://github.com/Sandrocamargo/biostatistics/blob/master/datasets/rl-milho.txt).

Abra este código no seu google colab [clicando aqui](https://colab.research.google.com/github/Sandrocamargo/biostatistics/blob/master/python/bioe_06_Regressao_Linear_Milho.ipynb).

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

# Carga e inspeção dos dados

In [ ]:
# === 1. Leitura dos dados ===
dados = pd.read_csv("https://raw.githubusercontent.com/Sandrocamargo/biostatistics/refs/heads/master/datasets/rl-milho.txt", sep=r"\s+", header=0)
dados.info()
dados.head()

# Adaptação de um modelo de regressão linear

In [ ]:
dados["trat"] = dados["dose"].astype("category")

modelo = smf.ols("prod ~ dose", data=dados).fit()
print("=== Modelo de regressão simples ===")
print(modelo.summary())

# Análise de variância

In [ ]:
print("\n=== ANOVA usual ===")
anova_geral = sm.stats.anova_lm(modelo, typ=2)
print(anova_geral)

In [ ]:
# === 3. Decomposição da SQTratamento ===
# SQTratamento = SQRegressão + SQFalta de Ajuste
desd_modelo = smf.ols("prod ~ dose + trat", data=dados).fit()
anova_desd = sm.stats.anova_lm(desd_modelo, typ=2)
print("\n=== ANOVA com decomposição (dose + trat) ===")
print(anova_desd)

# Coeficiente de determinação (r²)
SQReg = anova_desd.loc["dose", "sum_sq"]
SQTrat = SQReg + anova_desd.loc["trat", "sum_sq"]
r2 = SQReg / SQTrat
print(f"\nCoeficiente de Determinação (r²) = {r2:.3f}")

# Análise gráfica

In [ ]:
# === 4. Ajuste final e gráfico ===
modelo_ajustado = modelo  # mesmo modelo linear simples

b0, b1 = modelo_ajustado.params
print(f"\nModelo ajustado: y = {b0:.5f} + {b1:.5f}x")

# Geração do gráfico
plt.figure(figsize=(8, 6))
plt.scatter(dados["dose"], dados["prod"], color="blue", label="Observações")

# linha de regressão
x_vals = pd.Series(sorted(dados["dose"]))
y_vals = b0 + b1 * x_vals
plt.plot(x_vals, y_vals, color="red", label="Ajuste linear")

plt.xlabel("Dose adubo nitrogenado (kg/ha)")
plt.ylabel("Produção de milho (kg/parcela)")
plt.title(f"R² ajustado = {modelo_ajustado.rsquared_adj:.4f}")
plt.legend(title=f"y = {b0:.5f} + {b1:.5f}x", loc="upper left")
plt.grid(True)
plt.tight_layout()
plt.show()